In [ ]:
## Plots (TODO)

device = 'STATION_CASE'
test = '2018-08_INT_STATION_TEST_SUMMER_HOLIDAYS'

min_date = '2018-08-03 00:00:00'
max_date = '2018-09-17 00:00:00'

# Assign by time-frames
freq_time = 2

# Select labels
list_channels = ['EXT_PM_1', 'EXT_PM_25', 'EXT_PM_10', 'TEMP']

## -----------
dataframePlot = pd.DataFrame()
dataframePlot = records.readings[test]['devices'][device]['data'].copy()
dataframePlot = dataframePlot[dataframePlot.index>min_date]
dataframePlot = dataframePlot[dataframePlot.index<max_date]

if freq_time == 6:
    labels = ['Morning','Afternoon','Evening', 'Night']
    len_labels = 4
elif freq_time == 12:
    labels = ['Morning', 'Evening']
    len_labels = 2
else:
    labels = [str(i) for i in np.arange(0, 24, freq_time)]
    len_labels = freq_time * len(labels)
    
vector_time = np.arange(0, 25, freq_time)

dataframePlot = dataframePlot.assign(session=pd.cut(dataframePlot.index.hour,
                                            vector_time,
                                            labels=labels))
# Group them by session
df_se = dataframePlot.groupby(['session']).mean()
df_se = df_se[list_channels]

# Calculate average
df_se_avg = df_se.mean(axis = 0)

display(df_se)

# Add additional columns
append_rel = '_AVG_REL'

list_all = []
list_all.append('session')
for column in list_channels:
    dataframePlot[column + append_rel] = dataframePlot[column]/df_se_avg[column]
    list_all.append(column)
    list_all.append(column + append_rel)

## Full dataframe
dataframePlot = dataframePlot[list_all]
dataframePlot.dropna()

## Dataframe by sessionb
dataframePlot_avg = dataframePlot.groupby(['session']).mean()

In [ ]:
### Heatmap

In [ ]:
# Plotly
import plotly as ply
import plotly.graph_objs as go
from plotly.widgets import GraphWidget
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Layout
import plotly.tools as tls

init_notebook_mode(connected=True)

import matplotlib.pyplot as plot

channel =  'EXT_PM_25'
colorscale = [[0, '#edf8fb'], [.3, '#b3cde3'],  [.6, '#8856a7'],  [1, '#810f7c']]

# Data
data = [
    go.Heatmap(
        z=dataframePlot[channel],
        x=dataframePlot.index.date,
        y=dataframePlot['session'],
        colorscale=colorscale,
    )
]

layout = go.Layout(
    title='Daily Pollutant',
    xaxis = dict(ticks=''),
    yaxis = dict(ticks='' , categoryarray=labels, autorange = 'reversed')
)

fig = go.Figure(data=data, layout=layout)
ply.offline.plot(fig)


In [ ]:
### Vertical Bar Plot

In [ ]:
import matplotlib.pyplot as plot
import plotly as ply
import plotly.graph_objs as go
from plotly import tools


channel_1 =  'EXT_PM_1' # Goes on the left side (x1)
channel_2 =  'TEMP' # Goes on the right side (x2)
relative = False

if relative:
    channel_1 = channel_1 + append_rel
    channel_2 = channel_2 + append_rel
    limits = ([0.5, 1.5], [0.5, 1.5])
    marks = ()
else:
    limits = ([0, 20], [15, 40]) # x1, x2
    marks = ([10, 'x'], [20, 'x'], [300, 'x2'], [400, 'x2'])

dict_shapes = [{'type': 'line', 
                      'x0': mark[0],
                      'y0': -1,
                      'x1': mark[0],
                      'y1': len_labels+1,
                      'xref': mark[1],
                      'line': {
                          'color': 'rgba(5, 0, 0, 0.8)',
                          'width': 1,
                          'dash': 'dot'
                      }}
               for mark in marks]

trace1 = go.Bar(
            x=dataframePlot_avg[channel_1],
            y=labels,
            orientation = 'h',
            xaxis = 'x',
            yaxis = 'y',
            name = channel_1,
            marker = dict(color = 'rgba(58, 78, 255, 0.6)',
                          line = dict(color = 'rgba(58, 78, 255, 1.0)', width = 2))
)

trace2 = go.Bar(
            x=dataframePlot_avg[channel_2],
            y=labels,
            orientation = 'h',
            xaxis = 'x2',
            yaxis='y',
            name = channel_2,
            marker = dict(color = 'rgba(58, 71, 80, 0.6)',
                          line = dict(color = 'rgba(58, 71, 80, 1.0)', width = 2))
)

data = [trace1, trace2]

layout = go.Layout(title='Daily average measurement (freq = {}h)'.format(freq_time),
                   xaxis = dict(domain=[0, 0.5], autorange='reversed', title = channel_1), # range=(limits[0][0], limits[0][1])),
                   xaxis2 = dict(domain=[0.5, 1], title = channel_2), # range=(limits[1][0], limits[1][1])),
                   yaxis = dict(range=(-1, len_labels+1), autorange='reversed'),
                   yaxis2 = dict(range=(-1, len_labels+1), autorange='reversed'),
                   bargap = 0)#,
                   #shapes = dict_shapes)

fig = go.Figure(data=data, layout=layout)
ply.offline.plot(fig)